# Fine-grained Arabic Named Entity Recognition

# Bi-LSTM Model

# Load Notebook Environment
Load the pretrained kernel and environment without having to rerun all cells.

In [ ]:
import dill
dill.load_session('notebookWorking_env.db')

# Import Libraries

In [ ]:
# Import the Abstract Syntax Tree (ast) module for parsing Python code
import ast

# Import the NumPy library and alias it as np
import numpy as np

# Import the codecs module for handling different character encodings
import codecs

# Import the train_test_split function from scikit-learn for splitting data into training and testing sets
from sklearn.model_selection import train_test_split

# Import the Sequential model and various layers from Keras for building neural networks
from keras.models import Sequential
from keras.layers import InputLayer, Embedding, LSTM, Dense, Activation

# Import the Adam optimizer from Keras for training neural networks
from keras.optimizers import Adam

# Import the pad_sequences function for padding sequences in Keras
from keras.preprocessing.sequence import pad_sequences

# Import the to_categorical function from Keras for one-hot encoding categorical variables
from keras.utils import to_categorical

# Read/Processes Training Data

In [2]:
sentences = []
labels = []

curr_sentence = []
curr_labels = []
final_lines = []
final = ""
with open("/Users/ziadhamwi/Desktop/TeacherStudent/TrainingData.txt", "r") as file:
    for line in file:
        if line != "\n":
            label, word = line.split()[:2]
            
            final_lines.append(f"{label} {word}\n")
            
            curr_sentence.append(word)
            curr_labels.append(label)
        else:
            final_lines.append("\n")
            sentences.append(curr_sentence)
            labels.append(curr_labels)
            curr_sentence = []
            curr_labels = []
            
final = "".join(final_lines)

print("DONE!")


DONE!


In [3]:
print(len(final))

9735011


In [4]:
new_format = []
sentence = []

output2023 = ["["]
lines = final.split('\n')
drop = len(final.split('\n'))

for i in range(drop):
    curr_line = lines[i]
    if len(curr_line) != 0:
        parts = curr_line.split()
        subentity = parts[0]
        word = parts[1]

        if word != ".":
            output2023.append(f"('{word}', '{subentity}')")
            if i != drop-1:  
                output2023.append(", ")
        else:
            output2023.append("('.', 'OUTSIDE')]\n[")
            
if output2023[-1] == "[":
    output2023.pop()
else:
    if output2023[-1] == ", ":
        output2023.pop()

output2023.append("]")

output2023 = "".join(output2023)


print("Done!")


Done!


In [5]:
training_sentences = []
for line in output2023[:-3].split("\n"):
    training_sentences.append(line+"\n")
    
print(len(training_sentences))


27200


# Read/Processes Testing Data

In [6]:
sentences = []
labels = []

curr_sentence = []
curr_labels = []
final_lines = []
final = ""
with open("/Users/ziadhamwi/Desktop/TeacherStudent/TestingData.txt", "r") as file:
    for line in file:
        if line != "\n":
            label, word = line.split()[:2]
            
            final_lines.append(f"{label} {word}\n")
            
            curr_sentence.append(word)
            curr_labels.append(label)
        else:
            final_lines.append("\n")
            sentences.append(curr_sentence)
            labels.append(curr_labels)
            curr_sentence = []
            curr_labels = []
            
final = "".join(final_lines)

print("DONE!")


DONE!


In [7]:
print(len(final))

306023


In [8]:
new_format = []
sentence = []

output2023 = ["["]
lines = final.split('\n')
drop = len(final.split('\n'))

for i in range(drop):
    curr_line = lines[i]
    if len(curr_line) != 0:
        parts = curr_line.split()
        subentity = parts[0]
        word = parts[1]

        if word != ".":
            output2023.append(f"('{word}', '{subentity}')")
            if i != drop-1:
                output2023.append(", ")
        else:
            output2023.append("('.', 'OUTSIDE')]\n[")
            
if output2023[-1] == "[":
    output2023.pop()
else:
    if output2023[-1] == ", ":
        output2023.pop()

output2023.append("]")

output2023 = "".join(output2023)


print("Done!")


Done!


In [9]:
testing_sentences = []
for line in output2023[:-3].split("\n"):
    testing_sentences.append(line+"\n")
    
print(len(testing_sentences))


856


# Preview data

In [10]:
# Display a few rows of the dataset
num_rows = 5  # Number of rows to display
print("\n\nTraining Set")
for i in range(num_rows):
    print(training_sentences[i])

print("\n\nTesting Set")
for i in range(num_rows):
    print(testing_sentences[i])



Training Set
[('إنجلترا', 'B-Country'), ('أو', 'OUTSIDE'), ('إنكلترا', 'B-Country'), ('أو', 'OUTSIDE'), ('إنكلترة', 'B-Country'), ('بالإنجليزية', 'B-Language'), ('إنگلاند', 'OUTSIDE'), ('هي', 'OUTSIDE'), ('أكبر', 'OUTSIDE'), ('دولة', 'OUTSIDE'), ('في', 'OUTSIDE'), ('المملكة', 'B-Country'), ('المتحدة', 'I-Country'), ('وتشترك', 'OUTSIDE'), ('في', 'OUTSIDE'), ('الحدود', 'OUTSIDE'), ('البرية', 'OUTSIDE'), ('مع', 'OUTSIDE'), ('اسكتلندا', 'B-Country'), ('في', 'OUTSIDE'), ('الشمال', 'OUTSIDE'), ('وويلز', 'OUTSIDE'), ('في', 'OUTSIDE'), ('الغرب', 'OUTSIDE'), ('والبحر', 'B-Body_Of_Water'), ('الأيرلندي', 'I-Body_Of_Water'), ('في', 'OUTSIDE'), ('الشمال', 'OUTSIDE'), ('الغربي', 'OUTSIDE'), ('وبحر', 'B-Body_Of_Water'), ('الكلت', 'I-Body_Of_Water'), ('في', 'OUTSIDE'), ('الجنوب', 'OUTSIDE'), ('الغربي', 'OUTSIDE'), ('وبحر', 'OUTSIDE'), ('الشمال', 'OUTSIDE'), ('في', 'OUTSIDE'), ('الشرق', 'OUTSIDE'), ('وتفصلها', 'OUTSIDE'), ('القناة', 'OUTSIDE'), ('الإنجليزية', 'OUTSIDE'), ('عن', 'OUTSIDE'), ('القارة',

# NLP Data Preprocessing Functions for Neural Network Training
The code defines several functions for preprocessing natural language processing (NLP) data to be used in training and testing neural networks. The get_words function extracts unique words from a collection of sentences, while get_tags extracts unique tags from a set of sentence tags. The get_train_sentences_x and get_test_sentences_x functions convert sentences into index sequences for training and testing data, respectively, using a word-to-index mapping. Similarly, the get_train_tags_y and get_test_tags_y functions convert tags into index sequences for training and testing data, utilizing a tag-to-index mapping. These functions play a crucial role in preparing NLP data for machine learning models by transforming textual information into a format suitable for input to neural networks, addressing the need for numerical representation and handling out-of-vocabulary words.

In [11]:
def get_words(sentences):
    return set(word for sentence in sentences for word in sentence)

def get_tags(sentence_tags):
    return set(tag for tags in sentence_tags for tag in tags)

def get_train_sentences_x(sentences, word2index):
    return [[word2index[word] for word in sentence] for sentence in sentences]

def get_test_sentences_x(sentences, word2index):
    return [[word2index.get(word, word2index['-OOV-']) for word in sentence] for sentence in sentences]

def get_train_tags_y(tags, tag2index):
    return [[tag2index[tag] for tag in tag_sequence] for tag_sequence in tags]

def get_test_tags_y(tags, tag2index):
    return [[tag2index[tag] for tag in tag_sequence] for tag_sequence in tags]

# Converting Logits to Tokens in Natural Language Processing
The code presents a function named logits_to_tokens that serves to convert model predictions in the form of logits into their corresponding tokens or tags. The function takes two parameters: predictions, a list of predicted logits, and index_to_tag, a mapping associating index values with specific tags. Utilizing NumPy, the function identifies the index with the maximum value in each prediction and retrieves the corresponding tag from the provided mapping. The outcome is a list of tokens, where each token represents the predicted tag for the respective input logits. This function is commonly employed in natural language processing applications, particularly during the final step of converting neural network output into human-readable token predictions.

In [12]:
# Function to convert logits to tokens
def logits_to_tokens(predictions, index_to_tag):
    return [index_to_tag[np.argmax(prediction)] for prediction in predictions]

# Extracting Sentences and Tags from Tagged Sentences in Natural Language Processing
This code snippet processes a list of tagged sentences, extracting individual sentences and their associated tags. It iterates through each tagged sentence, utilizing ast.literal_eval to safely evaluate the expression and unpack the sentence and tags. The extracted sentence and tags are converted to lists and appended to the sentences and sentence_tags lists, respectively. To handle potential syntax errors during evaluation, a SyntaxError exception is caught, and the problematic line is printed for further inspection. This type of data extraction is typical in natural language processing tasks, where labeled sentences need to be separated into input sequences and corresponding tags for training machine learning models.

In [13]:
# Extract sentences and tags from tagged sentences -> Training Sentences/Tags
sentences_training, training_sentence_tags = [], []
for tagged_sentence in training_sentences:
    try:
        sentence, tags = zip(*ast.literal_eval(tagged_sentence.strip()))
        sentences_training.append(list(sentence))  # Convert to list to handle Arabic characters
        training_sentence_tags.append(list(tags))
    except SyntaxError:
        print(f"SyntaxError -Training- in line: {tagged_sentence.strip()}")
        

# Extract sentences and tags from tagged sentences -> Testing Sentences/Tags
sentences_testing, testing_sentence_tags = [], []
for tagged_sentence in testing_sentences:
    try:
        sentence, tags = zip(*ast.literal_eval(tagged_sentence.strip()))
        sentences_testing.append(list(sentence))  # Convert to list to handle Arabic characters
        testing_sentence_tags.append(list(tags))
    except SyntaxError:
        print(f"SyntaxError -Testing- in line: {tagged_sentence.strip()}")
        
        
train_sentences = sentences_training
train_tags = training_sentence_tags

test_sentences = sentences_testing
test_tags = testing_sentence_tags

In [ ]:
train_sentences = sentences_training
train_tags = training_sentence_tags

test_sentences = sentences_testing
test_tags = testing_sentence_tags

# Obtaining Unique Words and Tags from NLP Data
This code snippet utilizes the functions get_words and get_tags to obtain unique words and tags from the sentences and sentence_tags lists, respectively. The resulting sets of unique words and tags are stored in the variables words and tags. These unique elements play a pivotal role in preparing and processing natural language processing data for machine learning tasks, providing the foundation for creating mappings and numerical representations essential for model training.

In [14]:
# Get unique words and tags
words = get_words(train_sentences)
tags = get_tags(train_tags)

# Creating Word-to-Index and Tag-to-Index Dictionaries for NLP Data
This code snippet creates two dictionaries, word2index and tag2index, serving as mappings for words and tags to numerical indices. The word2index dictionary assigns indices to unique words, starting from 2, and includes special tokens '-PAD-' and '-OOV-' with indices 0 and 1, respectively. Similarly, the tag2index dictionary assigns indices to tags, starting from 1, with '-PAD-' assigned index 0. These dictionaries are essential for converting words and tags into numerical representations, a crucial step in preparing input data for neural network training in natural language processing applications.

In [15]:
# Create word-to-index and tag-to-index dictionaries
word2index = {w: i + 2 for i, w in enumerate(words)}
word2index['-PAD-'] = 0
word2index['-OOV-'] = 1

tag2index = {t: i + 1 for i, t in enumerate(tags)}
tag2index['-PAD-'] = 0


# Converting Sentences and Tags to Index Sequences for NLP Model Training
This code converts the sentences and corresponding tags from the training and testing sets into index sequences using functions tailored for training and testing data. The resulting sequences, stored in variables such as train_sentences_x and test_tags_y, represent numerical indices based on the word-to-index and tag-to-index dictionaries (word2index and tag2index). These numerical representations are crucial for preparing the input data for training and evaluating neural network models in natural language processing applications, allowing the model to work with numerical values rather than raw text.

In [16]:
# Convert sentences and tags to index sequences
train_sentences_x = get_train_sentences_x(train_sentences, word2index)
test_sentences_x = get_test_sentences_x(test_sentences, word2index)

train_tags_y = get_train_tags_y(train_tags, tag2index)
test_tags_y = get_test_tags_y(test_tags, tag2index)

# Padding Sequences for Uniform Length in NLP Model Training
This code snippet pads the sequences of sentences and tags to a common length, determined by the maximum length among sentences in both the training and testing sets. It uses the pad_sequences function from Keras to ensure uniform sequence lengths. The parameter padding='post' indicates that padding is added at the end of each sequence. This step is vital for preparing input data for neural network models, especially those requiring fixed-length input sequences, such as recurrent neural networks. Uniform sequence lengths facilitate efficient model training and processing of input data.

In [17]:
# Pad sequences to a common length
MAX_LENGTH = max(len(max(train_sentences_x, key=len)), len(max(test_sentences_x, key=len)))

train_sentences_x = pad_sequences(train_sentences_x, maxlen=MAX_LENGTH, padding='post')
test_sentences_x = pad_sequences(test_sentences_x, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

# Building and Compiling a Bidirectional LSTM Model for Sequence Labeling
This code segment builds and compiles a Bidirectional Long Short-Term Memory (BiLSTM) model using the Keras library. The model includes an input layer with a specified shape, an embedding layer for word representations, a Bidirectional LSTM layer with 128 units and return sequences set to True for sequence labeling tasks, and a dense layer with the number of tags as output units. The softmax activation function is applied for multi-class classification. The model is compiled with categorical crossentropy as the loss function, the Adam optimizer with a learning rate of 0.001, and accuracy as the evaluation metric. This architecture is commonly employed in natural language processing tasks, particularly for sequence labeling.

In [18]:
from keras.layers import Bidirectional

model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,)))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(len(tag2index)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

# Training


In [26]:
# Train the model
history = model.fit(train_sentences_x, to_categorical(train_tags_y, len(tag2index)), batch_size=32, epochs=5, verbose=1)

Epoch 1/5
850/850 [==============================] - 323s 380ms/step - loss: 0.0154 - accuracy: 0.9953
Epoch 2/5
850/850 [==============================] - 353s 415ms/step - loss: 0.0124 - accuracy: 0.9959
Epoch 3/5
850/850 [==============================] - 345s 406ms/step - loss: 0.0105 - accuracy: 0.9963
Epoch 4/5
850/850 [==============================] - 317s 372ms/step - loss: 0.0093 - accuracy: 0.9965
Epoch 5/5
850/850 [==============================] - 306s 360ms/step - loss: 0.0084 - accuracy: 0.9967


# Model Anatomy


In [27]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 289, 128)          7301120   
                                                                 
 bidirectional (Bidirection  (None, 289, 256)          263168    
 al)                                                             
                                                                 
 dense (Dense)               (None, 289, 213)          54741     
                                                                 
 activation (Activation)     (None, 289, 213)          0         
                                                                 
Total params: 7619029 (29.06 MB)
Trainable params: 7619029 (29.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Save Model


In [28]:
model.save("BiLSTM.h5")

/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Dump Notebook Environment
Dump current kernel and notebook environment to the file below so you can quickly load it next time you open the notebook.

In [ ]:
import dill
dill.dump_session('notebookWorking_env.db')

In [31]:
from sklearn.metrics import confusion_matrix
import numpy as np

predictions = model.predict(test_sentences_x)

predicted_labels = np.argmax(predictions, axis=2)

true_labels_flat = test_tags_y.flatten()
predicted_labels_flat = predicted_labels.flatten()

non_pad_elements = true_labels_flat != 0

conf_matrix = confusion_matrix(true_labels_flat[non_pad_elements], predicted_labels_flat[non_pad_elements])

print("Confusion Matrix:")
print(conf_matrix)


27/27 [==============================] - 3s 101ms/step
Confusion Matrix:
[[ 3  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0 28]]


In [34]:
from sklearn.metrics import classification_report
from tabulate import tabulate

non_pad_elements = true_labels_flat != 0

report = classification_report(true_labels_flat[non_pad_elements], 
                               predicted_labels_flat[non_pad_elements], 
                               labels=np.unique(predicted_labels_flat[non_pad_elements]))

print("Classification Report:")
# print(report)


array_output = report.split()

for i in range(4, len(array_output)-20, 5):
    id = array_output[i]
    
    desired_value = int(id)
    desired_key = None

    for key, value in tag2index.items():
        if value == desired_value:
            desired_key = key
            break
    array_output[i] = desired_key
    
# print(array_output)
array_output = array_output[9:]

# print(len(array_output))
for i in range(len(array_output) - 1, -1, -1):
    if array_output[i] == "avg":
        array_output[i-1] += "_avg"
        del array_output[i]
# print(array_output)


table = [array_output[i:i+5] for i in range(0, len(array_output), 5)]

print(tabulate(table[0:], headers=['Precision', 'Recall', 'F1-score', 'Support'], tablefmt='pretty'))



Classification Report:
+---------------------------+-----------+--------+----------+---------+
|                           | Precision | Recall | F1-score | Support |
+---------------------------+-----------+--------+----------+---------+
|        I-Newspaper        |   1.00    |  1.00  |   1.00   |    1    |
|         I-Artist          |   0.62    |  0.28  |   0.38   |   18    |
|        I-Military         |   0.57    |  0.29  |   0.38   |   14    |
|        B-Computer         |   0.33    |  0.03  |   0.05   |   38    |
|        I-Language         |   0.00    |  0.00  |   0.00   |    8    |
|    B-Religious_Leader     |   1.00    |  0.22  |   0.36   |    9    |
|        I-Ethnicity        |   1.00    |  1.00  |   1.00   |    1    |
|          I-City           |   0.67    |  0.22  |   0.33   |    9    |
|        B-Currency         |   0.75    |  0.79  |   0.77   |   19    |
|         B-Airport         |   1.00    |  1.00  |   1.00   |    4    |
|   B-Educational_Degree    |   0.00    |

/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Testing the Model

This code turns the model into a tag-wielding champion! Equipped with a custom function, it takes any sentence and predicts the appropriate tag for each word, like adding punctuation marks to complete a thought. Watch as it analyzes a sample sentence and unveils its predicted tags, revealing the model's real-world potential to understand and structure language on the fly!

In [36]:
# Function to predict tags for a given sentence
def predict_tags(sentence, model, word2index, index2tag):

    sentence_x = [word2index.get(word, word2index['-OOV-']) for word in sentence]

    sentence_x = pad_sequences([sentence_x], maxlen=MAX_LENGTH, padding='post')

    predictions = model.predict(sentence_x)

    predicted_tags = logits_to_tokens(predictions[0], index2tag)

    return predicted_tags

test_sentence = ['كيرلا', 'هي', 'ولاية', 'هندية', 'تقع', 'على', 'ساحل', 'مليبار', 'الهندي', '،', 'أُنشئت', 'في', '1', 'نوفمبر', '1956', 'بعد', 'إقرار', 'قانون', 'إعادة', 'تنظيم', 'الولايات', '،', 'حيث', 'جمعت', 'بين', 'المناطق', 'الناطقة', 'باللغة', 'المالايالامية', 'من', 'مناطق', 'كوشين', 'ومليبار', 'وجنوب', 'كانارا', 'وترافانكور', '،', 'وتبلغ', 'مساحتها', '38,863', 'كيلومتر', 'مربع', '(15,005', 'ميل2)', 'وهي', 'الولاية', 'الهندية', 'الحادية', 'والعشرون', 'من', 'حيث', 'المساحة', '،', 'تحدها', 'ولاية', 'كارناتاكا', 'من', 'الشمال', 'والشمال', 'الشرقي', '،', 'وتاميل', 'نادو', 'من', 'الشرق', 'والجنوب', '،', 'وبحر', 'لكديف', 'من', 'الغرب', '.']



index2tag = {i: t for t, i in tag2index.items()}

predicted_tags = predict_tags(test_sentence, model, word2index, index2tag)

print("Original Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags)

1/1 [==============================] - 0s 42ms/step
Original Sentence: ['كيرلا', 'هي', 'ولاية', 'هندية', 'تقع', 'على', 'ساحل', 'مليبار', 'الهندي', '،', 'أُنشئت', 'في', '1', 'نوفمبر', '1956', 'بعد', 'إقرار', 'قانون', 'إعادة', 'تنظيم', 'الولايات', '،', 'حيث', 'جمعت', 'بين', 'المناطق', 'الناطقة', 'باللغة', 'المالايالامية', 'من', 'مناطق', 'كوشين', 'ومليبار', 'وجنوب', 'كانارا', 'وترافانكور', '،', 'وتبلغ', 'مساحتها', '38,863', 'كيلومتر', 'مربع', '(15,005', 'ميل2)', 'وهي', 'الولاية', 'الهندية', 'الحادية', 'والعشرون', 'من', 'حيث', 'المساحة', '،', 'تحدها', 'ولاية', 'كارناتاكا', 'من', 'الشمال', 'والشمال', 'الشرقي', '،', 'وتاميل', 'نادو', 'من', 'الشرق', 'والجنوب', '،', 'وبحر', 'لكديف', 'من', 'الغرب', '.']
Predicted Tags: ['B-County', 'OUTSIDE', 'OUTSIDE', 'B-Nationality', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'B-Nationality', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'B-Date', 'I-Date', 'I-Date', 'OUTSIDE', 'OUTSIDE', 'B-Law', 'I-Law', 'I-Law', 'I-Law', 'I-Law', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'OUT